In [1]:
! pip install --quiet tqdm

In [2]:
import pandas as pd
import os
from lib.preprocess import preprocessor
from tqdm import tnrange, tqdm_notebook
import threading 

In [3]:
def process_readmes(df, pool):
    df = df[df.content.notna()].reset_index()
    processed = pool.map(preprocessor, df.content)
    df['content'] = processed
    df = df[df.content.notna()].reset_index()
    df['repo_owner'] = df.repo_name.map(lambda s: s.split('/')[0])
    return df

In [26]:
_claims_prepro('slkfklj sdlkj WO 978/98098')

In [60]:
num = re.compile(r'[0-9]+')

def _claims_prepro(s):
    s = preprocessor(s)
    s = re.sub(num, '', s) if s else None
    return s



In [43]:
from  multiprocessing import Pool

In [63]:
class ClaimsProcessor():
    def __init__(self, inpath, outpath):
        self.inpath = inpath
        self.outpath = outpath
        
    def process_claims(self,  
                       f, 
                       id_key='application_number_formatted', 
                       key='description_text', 
                       compression = 'gzip'):
                
        df = pd.read_csv('{}/{}'.format(self.inpath, f), compression=compression)
    
        df = df[df[key].notna()].reset_index(drop=True)
        processed = df[key].map(_claims_prepro)
        df['content'] = processed
        df = (df[(df.content.notna()) & (df[id_key].notna())]
              .reset_index(drop=True)
              .drop(key, 1))
    
        df.to_csv('{}/{}'.format(self.outpath,f), header=False, index=False)
        print('Written: {}'.format(f))

In [ ]:
pool = Pool()
inpath = 'oss_bigquery_exports/patent-descriptions'
files = get_files(inpath)
fn = ClaimsProcessor(inpath, 'patent-processed').process_claims
pool.map(fn, files)
pool.close()
pool.join()

Written: 000000000124
Written: 000000000052
Written: 000000000138
Written: 000000000029
Written: 000000000222
Written: 000000000208
Written: 000000000229
Written: 000000000203
Written: 000000000036
Written: 000000000231
Written: 000000000263
Written: 000000000248
Written: 000000000240
Written: 000000000019
Written: 000000000363
Written: 000000000546
Written: 000000000416
Written: 000000000307
Written: 000000000422
Written: 000000000419
Written: 000000000313
Written: 000000000535
Written: 000000000424
Written: 000000000478
Written: 000000000508
Written: 000000000320
Written: 000000000487
Written: 000000000390
Written: 000000000455
Written: 000000000473
Written: 000000000288
Written: 000000000256
Written: 000000000000
Written: 000000000062
Written: 000000000193
Written: 000000000012
Written: 000000000181
Written: 000000000065
Written: 000000000259
Written: 000000000084
Written: 000000000251
Written: 000000000117
Written: 000000000197
Written: 000000000182
Written: 000000000113
Written: 0

In [44]:
get_files = lambda inpath: list(os.walk(inpath))[0][2]

def preprocess(outfile, inpath, fn, id_key, compression=None):
    pool = Pool()
    files = get_files(inpath)

    with open(outfile, 'w') as f:
        for df in dfs:
            df = pool.map(fn, dfs)
            for i,c in zip(df[id_key], df.content):
                if i:
                    f.write('{},{}\n'.format(i,c))
    pool.close()
    pool.join()

In [ ]:
preprocess('patent_descriptions_lookup.csv',  'oss_bigquery_exports/patent-descriptions', 
                                                   process_claims, 
                                                   'application_number_formatted', 'gzip')

## Run for different datas

In [ ]:
preprocess('readme_lookup.csv', 'oss_bigquery_exports/readmes', process_readmes, 'id')

### Treat owner like collaborative filtering? 

In [101]:
by_owner = df.groupby('repo_owner').apply(lambda df: ' \t '.join(df.content))

In [102]:
with open('by_owner.txt', 'w') as f:
    for c in by_owner:
        f.write(c + '\n')